In [1]:
# pip install boto3 
import secret, time, boto3 # https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html

def AWS_Session(region_name): 
    aws_access_key_id     = secret.aws_access_key_id
    aws_secret_access_key = secret.aws_secret_access_key   
    session = boto3.Session(region_name           = region_name          , 
                            aws_access_key_id     = aws_access_key_id    , 
                            aws_secret_access_key = aws_secret_access_key)    
    return session

session = AWS_Session(region_name='us-east-1')    

class S3:
    def __init__(self):  
        self.client = session.client('s3')
        self.Buckets = {}
        
    def add_new_bucket(self, bucket_name):
        bucket = session.resource('s3').Bucket(bucket_name)
        self.Buckets.update({bucket_name:bucket}) 
        return True

s3 = S3()

### Create a publically accessible bucket

In [2]:
myBucketName = 'some1unused1name' # Only Alphanumericals 
region_name  = 'ap-northeast-2'

In [3]:
response = s3.client.create_bucket( 
    Bucket=myBucketName,
    CreateBucketConfiguration={
        'LocationConstraint': region_name
    },
) 

In [4]:
s3.add_new_bucket(myBucketName) 

True

### Upload a file

In [5]:
file_content = b'''<html xmlns="http://www.w3.org/1999/xhtml" >
<head>
    <title>My Website Home Page</title>
</head>
<body>
  <h1>Welcome to my website</h1>
  <p>Now hosted on Amazon S3!</p>
</body>
</html>'''

In [6]:
import hashlib, base64   
md5=lambda x: base64.b64encode(hashlib.md5(x).digest()).decode()   

In [7]:
response = s3.client.put_object(
    ACL          = 'public-read',
    Body         = file_content,
    Bucket       = myBucketName, 
    ContentMD5   = md5(file_content), 
    ContentType  = 'text/html',
    Key          = 'index.html',
    StorageClass = 'STANDARD'
)

### Confirm we have uploaded an object

In [8]:
print('List of files in our bucket:')
for obj in s3.Buckets[myBucketName].objects.all():
    print('  ',obj.key)

List of files in our bucket:
   index.html


### Enable the static websute hosting

In [9]:
response = s3.client.put_bucket_website(
    Bucket=myBucketName,
    WebsiteConfiguration={  
        'IndexDocument': {
            'Suffix': 'index.html'
        }
    }
)

### Visit out new website

In [10]:
print(f'http://{myBucketName}.s3-website.{region_name}.amazonaws.com/')

http://some1unused1name.s3-website.ap-northeast-2.amazonaws.com/


### Disable static website hosting

In [ ]:
_ = s3.client.delete_bucket_website(Bucket=myBucketName)

### Empty the bucket

In [ ]:
_ = s3.Buckets[myBucketName].objects.all().delete()

### Delete the bucket

In [ ]:
_ = s3.client.delete_bucket(Bucket=myBucketName) 